In [95]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader


In [110]:
class Configs:
    def __init__(self, n_epochs=5, train_batch_size=64, test_batch_size=1000, learning_rate=0.01, momentum=0.8):
        torch.manual_seed(9)
        self.transformer = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
        self.n_epochs = n_epochs
        self.train_batch_size = train_batch_size
        self.test_batch_size = test_batch_size
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.log_interval = 10

        self.train_configs = {'batch_size': self.train_batch_size}
        self.test_configs = {'batch_size': self.test_batch_size}
        if torch.cuda.is_available():
            self.device = torch.device("cuda:0")
            cuda_configs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
            self.train_configs.update(cuda_configs)
            self.test_configs.update(cuda_configs)

        else:
            self.device = torch.device("cpu")
    
    def get_train_kwargs(self):
        return self.train_configs
    
    def get_test_kwargs(self):
        return self.test_configs

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        # output = F.softmax(x, dim=1)
        output = x
        return output

def train(args, model, train_loader, optimizer, epoch, criterion):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(args.device), target.to(args.device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()
                )
            )

def validate(model, configs, val_loader, criterion):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(configs.device), target.to(configs.device)
            output = model(data)
            val_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        val_loss /= len(val_loader.dataset)

    print(f'\nValidation Average Loss: {val_loss:.4f}; Validation Average Accuracy: {correct}/{len(val_loader.dataset)} ({100. * correct / len(val_loader.dataset):.0f}%)\n')

def inference(model, configs, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(configs.device), target.to(configs.device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)

    print(f'\nTest Average Loss: {test_loss:.4f};Test Average Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [111]:
configs = Configs(n_epochs=3, train_batch_size=64, test_batch_size=1000, learning_rate=0.01, momentum=0.5)
mnist_train = datasets.MNIST(root='./datasets', train=True, download=True, transform=configs.transformer)
mnist_test = datasets.MNIST(root='./datasets', train=False, download=True, transform=configs.transformer)
train_loader = DataLoader(mnist_train, **configs.get_train_kwargs())
test_loader = DataLoader(mnist_test, **configs.get_test_kwargs())
model = Net().to(configs.device)
criterion = CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=configs.learning_rate)
for epoch in range(configs.n_epochs):
    train(configs, model, train_loader, optimizer, epoch, criterion)
    validate(model, configs, test_loader, criterion)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.339320
Train Epoch: 0 [640/60000 (1%)]	Loss: 1.826767
Train Epoch: 0 [1280/60000 (2%)]	Loss: 1.577013
Train Epoch: 0 [1920/60000 (3%)]	Loss: 1.287614
Train Epoch: 0 [2560/60000 (4%)]	Loss: 1.060851
Train Epoch: 0 [3200/60000 (5%)]	Loss: 1.025056
Train Epoch: 0 [3840/60000 (6%)]	Loss: 0.840179
Train Epoch: 0 [4480/60000 (7%)]	Loss: 0.786345
Train Epoch: 0 [5120/60000 (9%)]	Loss: 0.908402
Train Epoch: 0 [5760/60000 (10%)]	Loss: 0.757845
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.656764
Train Epoch: 0 [7040/60000 (12%)]	Loss: 0.687619
Train Epoch: 0 [7680/60000 (13%)]	Loss: 0.690501
Train Epoch: 0 [8320/60000 (14%)]	Loss: 0.631172
Train Epoch: 0 [8960/60000 (15%)]	Loss: 0.517873
Train Epoch: 0 [9600/60000 (16%)]	Loss: 0.555379
Train Epoch: 0 [10240/60000 (17%)]	Loss: 0.671723
Train Epoch: 0 [10880/60000 (18%)]	Loss: 0.477449
Train Epoch: 0 [11520/60000 (19%)]	Loss: 0.677540
Train Epoch: 0 [12160/60000 (20%)]	Loss: 0.610699
Train Epoch: 0 [12800/60000 (